<a href="https://colab.research.google.com/github/amannarsaria4/fast.ai/blob/master/Lesson%203/(MyCode)lesson3_planet_ipynb.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

##Datasets

In [ ]:
tfms = get_transforms(flip_vert = True, max_lighting = 0.1, max_zoom = 1.05, max_warp = 0.)

In [ ]:
src = ImageList.from_csv(path, train_v2.csv, folder = 'train-jpg', suffix = '.jpg').split_by_rand_pct(0.2).label_from_df(label_delim = ' ')

In [ ]:
data = src.transform(tfms, size).databunch().normalize(imagenet_stats)

##Model

In [ ]:
acc_02 = partial(accuracy_thresh, thresh = 0.2)
f_score = partial(fbeta, thresh = 0.2)
learn = cnn_learner(data, models.resnet50, metrics = [acc_02, f_score])

In [ ]:
learn.lr_find()
learn.recorder.plot()

In [ ]:
learn.fit_one_cycle(4, max_lr = 0.01)
learn.save('stage-1-rn50')

In [ ]:
learn.unfreeze()
learn.lr_find()
learn.recorder.plot()


In [ ]:
learn.fit_one_cycle(6, max_lr = slice(1e-5,1e-4))

In [ ]:
learn.save('stage-2-rn50')

##Large Model

In [ ]:
data = src.transform(tfms, size = 256).databunch().normalize(imagenet_stats)

In [ ]:
learn = cnn_learner(data, models.resnet50, metrics=[acc_02, f_score])
learn.load('stage-2-rn50')

In [ ]:
learn.lr_find()
learn.recorder.plot()


In [ ]:
learn.fit_one_cycle(5, lr = 0.01)

In [ ]:
learn.save('stage-1-rn50-big')

In [ ]:
learn.unfreeze()

In [ ]:
learn.lr_find()
learn.recorder.plot()

In [ ]:
learn.fit_one_cycle(5,slice(1e-5, 1e-4))

In [ ]:
learn.recoder.plot_losses()

In [ ]:
learn.save('stage-2-rn50-big')